In [1]:
import keras

c:\python3.6\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np
import pickle as pkl
import datetime
from skimage import exposure

# Load Cifar10 dataset

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

In [5]:
print('The training set has {0} samples with dimentions {1}.'.format(x_train.shape[0], x_train.shape[1:]))
print('The test set has {0} samples with dimentions {1}.'.format(x_test.shape[0], x_test.shape[1:]))

The training set has 50000 samples with dimentions (32, 32, 3).
The test set has 10000 samples with dimentions (32, 32, 3).


In [6]:
NUM_CLASSES = 10
IMG_ROWS, IMG_COLS = 32, 32
BATCH_SIZE = 512
EPOCHS = 20
VAL_SPLIT = .2
LEARNING_RATE = 1e-2
SGD_MOMENTUM = .9

# Preprocess data

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [8]:
x_train -= np.mean(x_train)
x_test -= np.mean(x_test)

In [9]:
x_train /= np.std(x_train)
x_test /= np.std(x_test)

In [10]:
y_train = keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

In [11]:
validation_split = int((1-VAL_SPLIT)*len(x_train))
x_train, x_val = x_train[:validation_split], x_train[validation_split:]
y_train, y_val = y_train[:validation_split], y_train[validation_split:]

In [12]:
def contrast_stretching(img):
    p2, p98 = np.percentile(img, (2, 98))
    return exposure.rescale_intensity(img, in_range=(p2, p98))

In [13]:
datagen = ImageDataGenerator(preprocessing_function=contrast_stretching)

# Build model

In [14]:
input_tensor = Input(shape=(IMG_ROWS, IMG_COLS, 3))

In [15]:
base_model = VGG16(include_top=False, input_tensor=input_tensor)

58892288/58889256 [==============================] - 9s 0us/step


In [16]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [17]:
vgg16 = Model(inputs=base_model.input, outputs=predictions)

In [18]:
vgg16.compile(optimizer=SGD(lr=LEARNING_RATE, momentum=SGD_MOMENTUM), loss='categorical_crossentropy', metrics=['accuracy'])

# Train model

In [19]:
now = datetime.datetime.now()
tensorflow_log_dir = "Graph/" + now.strftime("%Y%m%d-%H%M%S") + "/"

In [20]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorflow_log_dir, histogram_freq=0,  
          write_graph=False, write_images=False)

In [21]:
vgg16.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                    steps_per_epoch=len(x_train) / BATCH_SIZE, 
                    validation_data=ImageDataGenerator().flow(x_val, y_val, batch_size=BATCH_SIZE),
                    validation_steps=len(x_val) / BATCH_SIZE,
                    epochs=EPOCHS, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/20
79/78 [==============================] - 82s 1s/step - loss: 2.2989 - acc: 0.1409 - val_loss: 2.3027 - val_acc: 0.1025
Epoch 2/20
50/78 [==================>...........] - ETA: 23s - loss: 2.3028 - acc: 0.1001

KeyboardInterrupt: 

In [22]:
vgg16.evaluate(x_test, y_test, verbose=1)

10000/10000 [==============================] - 9s 931us/step


[2.302720950317383, 0.1]